In [2]:
import numpy as np
from scipy.integrate import odeint

In [3]:
# reservoir_size = 10

In [4]:
# W = np.zeros((reservoir_size, reservoir_size))
# for i in range(reservoir_size):
#     j = (i+1) % reservoir_size
#     W[i, j] = 1.0

In [5]:
# W

In [6]:
# W = np.zeros((reservoir_size, reservoir_size))
# for i in range(reservoir_size):
#     # Connect to 3 forward neighbors
#     for offset in range(1, 4):
#         j = (i + offset) % reservoir_size
#         W[i, j] = 1.0
#     # Connect to 3 backward neighbors
#     for offset in range(1, 4):
#         j = (i - offset) % reservoir_size
#         W[i, j] = 1.0

In [7]:
# W

In [8]:
# p = 0.5  # rewiring probability

# W = np.zeros((reservoir_size, reservoir_size))
# for i in range(reservoir_size):
#     for offset in range(1, 4):
#         W[i, (i + offset) % reservoir_size] = 1.0
#         W[i, (i - offset) % reservoir_size] = 1.0

# for i in range(reservoir_size):
#     current_connections = np.where(W[i] == 1.0)[0]
#     for j in current_connections:
#         if np.random.rand() < p:
#             W[i, j] = 0.0
#             possible_nodes = list(set(range(reservoir_size)) - {i} - set(np.where(W[i] == 1.0)[0]))
#             if possible_nodes:
#                 new_j = np.random.choice(possible_nodes)
#                 W[i, new_j] = 1.0

In [9]:
# W

In [10]:
def lorenz_deriv(state, t, sigma=10.0, rho=28.0, beta=8.0/3.0):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x*(rho - z) - y
    dzdt = x*y - beta*z
    return [dxdt, dydt, dzdt]

def generate_lorenz_data(
    initial_state=[1.0, 1.0, 1.0],
    tmax=25.0,
    dt=0.01,
    sigma=10.0,
    rho=28.0,
    beta=8.0/3.0
):
    
    num_steps = int(tmax / dt)
    t_vals = np.linspace(0, tmax, num_steps+1)
    sol = odeint(lorenz_deriv, initial_state, t_vals, args=(sigma, rho, beta))
    return t_vals, sol

In [11]:
from sklearn.preprocessing import StandardScaler

In [28]:
tmax = 110
dt   = 0.01
t_vals, lorenz_traj = generate_lorenz_data(
    initial_state=[1.0,1.0,1.0],
    tmax=tmax,
    dt=dt
)

washout = 1000
t_vals = t_vals[washout:]
lorenz_traj = lorenz_traj[washout:]

# normalize
scaler = StandardScaler()
scaler.fit(lorenz_traj)
lorenz_traj = scaler.transform(lorenz_traj)

T_data = len(lorenz_traj)
print(f"Data length: {T_data}, from t=0..{tmax} with dt={dt}.")

n_test_steps = 3000

# train/test split
train_frac = 0.7
train_end = int(train_frac*(T_data-1))
train_input  = lorenz_traj[:train_end]
train_target = lorenz_traj[1:train_end+1]
test_input   = lorenz_traj[train_end:train_end+n_test_steps]
test_target  = lorenz_traj[train_end+1:train_end+n_test_steps+1]
print(f"Train size: {len(train_input)}  Test size: {len(test_input)}")

initial_in = test_input[0]

Data length: 10001, from t=0..110 with dt=0.01.
Train size: 7000  Test size: 3000


In [29]:
test_input.shape

(3000, 3)

In [30]:
t_vals

array([ 10.  ,  10.01,  10.02, ..., 109.98, 109.99, 110.  ])

In [31]:
num_steps = int(tmax / dt)
t_vals = np.linspace(0, tmax, num_steps+1)

In [32]:
t_vals[1]

0.01

In [33]:
np.linspace(0, 110, 1101)

array([0.000e+00, 1.000e-01, 2.000e-01, ..., 1.098e+02, 1.099e+02,
       1.100e+02])

In [34]:
test_target  = lorenz_traj[train_end+1:train_end+n_test_steps+1]

In [35]:
test_target.shape

(3000, 3)

In [36]:
test_target  = lorenz_traj[train_end+1:train_end+n_test_steps+1]

In [37]:
test_target.shape

(3000, 3)